<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Глубокое обучение. Часть 2
# Домашнее задание по теме "Механизм внимания"

Это домашнее задание проходит в формате peer-review. Это означает, что его будут проверять ваши однокурсники. Поэтому пишите разборчивый код, добавляйте комментарии и пишите выводы после проделанной работы.

В этом задании вы будете решать задачу классификации математических задач по темам (многоклассовая классификация) с помощью Transformer.

В качестве датасета возьмем датасет математических задач по разным темам. Нам необходим следующий файл:

[Файл с классами](https://docs.google.com/spreadsheets/d/1IMRxByfg7gjoZ5i7rxvuNDvSrbdOJOc-/edit?usp=drive_link&ouid=104379615679964018037&rtpof=true&sd=true)

In [ ]:
!ls -la /kaggle/input/no-problems/

In [ ]:
!ls -la /kaggle/working/

In [ ]:
import pandas as pd

# Задайте путь к вашему файлу
filepath = "/kaggle/input/no-problems/data_problems.csv"

# Читаем данные из файла
data = pd.read_csv(filepath)

# Просмотрим первые несколько строк данных
print(data.iloc[:0].head())


In [ ]:
# Получаем первый столбец по индексу
first_column = data.iloc[:, 2]

# Находим уникальные значения
unique_values = first_column.unique()

# Подсчитываем количество уникальных значений
num_unique_values = len(unique_values)

print(f"Количество уникальных значений в первом столбце: {num_unique_values}")
print(f"они сами: {unique_values}")



**Hint:** не перезаписывайте модели, которые вы получите на каждом из этапов этого дз. Они ещё понадобятся.

### Задание 1 (2 балла)

Напишите кастомный класс для модели трансформера для задачи классификации, использующей в качествке backbone какую-то из моделей huggingface.

Т.е. конструктор класса должен принимать на вход название модели и подгружать её из huggingface, а затем использовать в качестве backbone (достаточно возможности использовать в качестве backbone те модели, которые упомянуты в последующих пунктах)

In [ ]:
!pip install transformers

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel
from typing import Union, Dict

In [ ]:
class TransformerClassificationModel(nn.Module):
    def __init__(self, base_transformer_model: Union[str, nn.Module], num_labels: int):
        """
        Инициализация модели классификации на основе трансформера.
        
        Параметры:
        base_transformer_model (Union[str, nn.Module]): Название предобученной модели трансформера из huggingface
                                                        или экземпляр модели nn.Module.
        num_labels (int): Количество классов для классификации.
        """
        super(TransformerClassificationModel, self).__init__()
        if isinstance(base_transformer_model, str):
            self.backbone = AutoModel.from_pretrained(base_transformer_model)
        else:
            self.backbone = base_transformer_model
        
        # Добавляем дополнительные слои для классификации.
        self.classifier = nn.Linear(self.backbone.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None) -> Dict[str, torch.Tensor]:
        """
        Прямой проход модели.
        
        Параметры:
        input_ids (torch.Tensor): Тензор идентификаторов токенов.
        attention_mask (torch.Tensor, optional): Тензор масок внимания.
        token_type_ids (torch.Tensor, optional): Тензор типов токенов.

        Возвращает:
        Dict[str, torch.Tensor]: Словарь с логитами предсказаний классов.
        """
        # Пропагация входных данных через backbone.
        outputs = self.backbone(input_ids=input_ids, 
                                attention_mask=attention_mask, 
                                token_type_ids=token_type_ids)
        
        # Используем [CLS] токен для классификации.
        cls_token_state = outputs.last_hidden_state[:, 0, :]
        
        # Пропагация через классификационный слой.
        logits = self.classifier(cls_token_state)

        return {"logits": logits}


### Задание 2 (1 балл)

Напишите функцию заморозки backbone у модели (если необходимо, возвращайте из функции модель)

In [ ]:
def freeze_backbone_function(model: TransformerClassificationModel):
    # Перебираем все параметры в backbone модели
    for param in model.backbone.parameters():
        # Выключаем вычисление градиентов
        param.requires_grad = False

    # Возвращать модель необязательно, так как изменения произведены "на месте"
    return model

### Задание 3 (2 балла)

Напишите функцию, которая будет использована для тренировки (дообучения) трансформера (TransformerClassificationModel). Функция должна поддерживать обучение с замороженным и размороженным backbone.

In [ ]:
import copy
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

def train_transformer(model, data_loader, freeze_backbone=True, epochs=3, learning_rate=5e-5, num_training_steps=None):
    # Создаем полную копию модели для ее дообучения
    finetuned_model = copy.deepcopy(model)

    # Замораживаем или размораживаем backbone
    for param in finetuned_model.backbone.parameters():
        param.requires_grad = not freeze_backbone

    # Создаем оптимизатор, оптимизируя только параметры, требующие градиенты
    optimizer = AdamW(filter(lambda p: p.requires_grad, finetuned_model.parameters()), lr=learning_rate)

    # Предполагаем использование функции потерь CrossEntropyLoss для классификации
    criterion = torch.nn.CrossEntropyLoss()

    # Если определено количество шагов обучения, создаем планировщик для управления скоростью обучения
    scheduler = None
    if num_training_steps is not None:
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    finetuned_model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in data_loader:
            # Предполагается, что DataLoader возвращает словарь с ключами 'input_ids', 'attention_mask', 'labels'
            inputs = {k: v.to(finetuned_model.device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(finetuned_model.device)

            outputs = finetuned_model(**inputs)
            loss = criterion(outputs['logits'], labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if scheduler:
                scheduler.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}: Loss = {total_loss / len(data_loader)}")

    return finetuned_model


### Задание 4 (1 балл)

Проверьте вашу функцию из предыдущего пункта, дообучив двумя способами
*cointegrated/rubert-tiny2* из huggingface.

In [ ]:
from transformers import AutoModel, AutoConfig
import torch.nn as nn

class RubertTinyClassifier(nn.Module):
    def __init__(self, pretrained_model_name, num_labels):
        super(RubertTinyClassifier, self).__init__()
        self.backbone = AutoModel.from_pretrained(pretrained_model_name)
        self.classifier = nn.Linear(self.backbone.config.hidden_size, num_labels)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooler_output = outputs.pooler_output
        logits = self.classifier(pooler_output)
        return {'logits': logits}



In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import torch


In [ ]:
# Шаг 1: Загрузка данных
file_path = '/kaggle/input/no-problems/data_problems.csv'
df = pd.read_csv(file_path)

# Шаг 2: Преобразование категорий в числовой формат
unique_labels = df['Тема'].unique()
label_to_id = {label: id for id, label in enumerate(unique_labels)}
id_to_label = {id: label for label, id in label_to_id.items()}

df['label_id'] = df['Тема'].map(label_to_id)

In [ ]:
# Шаг 3: Создание класса Dataset
class MathProblemsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=512):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['Задача']
        self.labels = dataframe['label_id']
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        text = str(self.text[idx])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=False,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = inputs['input_ids'].flatten()
        attention_mask = inputs['attention_mask'].flatten()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }


In [ ]:
# Шаг 4: Токенизация
# Замените 'pretrained_model_name' на название модели, которую вы планируете использовать
pretrained_model_name = 'cointegrated/rubert-tiny2'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

# Создание экземпляра Dataset
dataset = MathProblemsDataset(df, tokenizer)

# Создание DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
NUM_LABELS = 11
# Инициализация и дообучение с замороженным backbone
rubert_tiny_transformer_model = RubertTinyClassifier(pretrained_model_name="cointegrated/rubert-tiny2", num_labels=NUM_LABELS)
rubert_tiny_finetuned_with_freezed_backbone = train_transformer(rubert_tiny_transformer_model, loader, freeze_backbone=True)

# Инициализация и полное дообучение
rubert_tiny_transformer_model = RubertTinyClassifier(pretrained_model_name="cointegrated/rubert-tiny2", num_labels=NUM_LABELS)
rubert_tiny_full_finetuned = train_transformer(rubert_tiny_transformer_model, loader, freeze_backbone=False)


In [ ]:
# Сохранение модели после обучения с замороженным backbone
model_path = "/kaggle/working/rubert_tiny_finetuned_with_freezed_backbone.pth"
torch.save(rubert_tiny_finetuned_with_freezed_backbone.state_dict(), model_path)

# Сохранение модели после полного обучения
model_path = "/kaggle/working/rubert_tiny_full_finetuned.pth"
torch.save(rubert_tiny_full_finetuned.state_dict(), model_path)

In [ ]:
# перед обучением 2й модели очистим видеко память
import torch
import gc

del rubert_tiny_transformer_model
del rubert_tiny_finetuned_with_freezed_backbone 
del rubert_tiny_full_finetuned
gc.collect()  # Сборка мусора в Python (освобождение памяти)
torch.cuda.empty_cache()  # Освобождение неиспользуемой памяти на GPU

### Задание 5 (1 балл)

Обучите *tbs17/MathBert* (с замороженным backbone и без заморозки), проанализируйте результаты. Сравните скоры с первым заданием. Получилось лучше или нет? Почему?

In [ ]:
from transformers import AutoModel, AutoConfig
import torch.nn as nn
import torch



In [ ]:
from transformers import AutoTokenizer

# Загрузка токенизатора для модели tbs17/MathBert
tokenizer = AutoTokenizer.from_pretrained("tbs17/MathBert")


In [ ]:
# from torch.utils.data import DataLoader

# Замените 'pretrained_model_name' на название новой модели, которую вы планируете использовать
pretrained_model_name = 'tbs17/MathBert'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

# Создание экземпляра Dataset
dataset = MathProblemsDataset(df, tokenizer)

# Создание DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:

class MathBertClassifier(nn.Module):
    def __init__(self, pretrained_model_name, num_labels):
        super(MathBertClassifier, self).__init__()
        self.backbone = AutoModel.from_pretrained(pretrained_model_name)
        self.classifier = nn.Linear(self.backbone.config.hidden_size, num_labels)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        pooler_output = outputs.pooler_output
        logits = self.classifier(pooler_output)
        return {'logits': logits}

In [ ]:
NUM_LABELS = 11  # Убедитесь, что это количество уникальных классов в вашем датасете

# Инициализация и дообучение с замороженным backbone
math_bert_model_frozen = MathBertClassifier(pretrained_model_name="tbs17/MathBert", num_labels=NUM_LABELS)
math_bert_finetuned_frozen = train_transformer(math_bert_model_frozen, loader, freeze_backbone=True)

# Инициализация и полное дообучение
math_bert_model_unfrozen = MathBertClassifier(pretrained_model_name="tbs17/MathBert", num_labels=NUM_LABELS)
math_bert_finetuned_unfrozen = train_transformer(math_bert_model_unfrozen, loader, freeze_backbone=False)


### Задание 6 (1 балл)

Напишите функцию для отрисовки карт внимания первого слоя для моделей из задания

In [ ]:
def draw_first_layer_attention_maps(attention_head_ids: List, text: str, model: TransformerClassificationModel):
    pass

### Задание 7 (1 балл)

Проведите инференс для всех моделей **ДО ДООБУЧЕНИЯ** на 2-3 текстах из датасета. Посмотрите на головы Attention первого слоя в каждой модели на выбранных текстах (отрисуйте их отдельно).

Попробуйте их проинтерпретировать. Какие связи улавливают карты внимания? (если в модели много голов Attention, то проинтерпретируйте наиболее интересные)

In [ ]:
### YOUR CODE IS HERE

### Задание 8 (1 балл)

Сделайте то же самое для дообученных моделей. Изменились ли карты внимания и связи, которые они улавливают? Почему?

In [ ]:
### YOUR CODE IS HERE